In [1]:

from ibmdbpy import IdaDataBase, IdaDataFrame

# @hidden_cell
# This connection object is used to access your data and contains your credentials.
# You might want to remove those credentials before you share your notebook.
idadb_b17dd4d9dff84b519a9e627f3bec50c4 = IdaDataBase(dsn='DASHDB;Database=BLUDB;Hostname=dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net;Port=50000;PROTOCOL=TCPIP;UID=sdc18363;PWD=g6tf3xjbqm+p2hxd')

sql_data = IdaDataFrame(idadb_b17dd4d9dff84b519a9e627f3bec50c4, 'SDC18363.PAXFEEDBACK').as_dataframe()
sql_data.head()

# You can close the database connection with the following code. Please keep the comment line with the @hidden_cell tag,
# because the close function displays parts of the credentials.
# @hidden_cell
# idadb_b17dd4d9dff84b519a9e627f3bec50c4.close()
# To learn more about the ibmdby package, please read the documentation: http://pythonhosted.org/ibmdbpy/


,Customer Type,Comments,Response,Categorisation,Sentiment,Sector,Flight No,Flight Date,Cabin Class,Seat No,Aircraft Version,Tail No,FFP Member,Gender,Age,Country
0,Passive,Amenity Kit: Comments,Très bien.,Compliment,Positive,MRU-CDG,NK0034,26/11/2018,ECONOMY,039E,359C,3BNBQ,NO,Male,26-34,France
1,Detractor,Amenity Kit: Comments,Not taking well kids,No adequate products,Negative,MRU-MAA,NK0746,26/11/2018,ECONOMY,None,343E,3BNBJ,NO,Male,35-44,Mauritius
2,Detractor,Amenity Kit: Comments,jusque dans le détail nous voyons la régressio...,Poor quality/ Very dissatisfied,Negative,BOM-MRU,NK0749,26/11/2018,ECONOMY,011C,343E,3BNBI,NO,Female,45-54,Mauritius
3,Detractor,Amenity Kit: Comments,I appreciate this kit in economy. It is a welc...,Compliment,Positive,SIN-MRU,NK0647,27/11/2018,ECONOMY,016B,332,3BNBM,NO,Female,55-64,Mauritius
4,Detractor,Amenity Kit: Comments,Nothing,None,Neutral,MRU-BOM,NK0748,27/11/2018,ECONOMY,026H,343C,3BNAU,YES,Male,45-54,India


In [2]:
# Data Inforamtion 
sql_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5347 entries, 0 to 5346
Data columns (total 16 columns):
Customer Type       5322 non-null object
Comments            5347 non-null object
Response            5334 non-null object
Categorisation      5300 non-null object
Sentiment           5179 non-null object
Sector              5347 non-null object
Flight No           5347 non-null object
Flight Date         5347 non-null object
Cabin Class         5347 non-null object
Seat No             4914 non-null object
Aircraft Version    5347 non-null object
Tail No             5317 non-null object
FFP Member          5317 non-null object
Gender              5224 non-null object
Age                 5185 non-null object
Country             5185 non-null object
dtypes: object(16)
memory usage: 668.5+ KB


In [3]:
# Drop null Records
sql_data.dropna(how='all',axis=0,inplace=True)
sql_data.shape

(5347, 16)

In [4]:
#upgrade Watson Packages
! pip install --upgrade "ibm-watson>=4.3.0"

Requirement already up-to-date: ibm-watson>=4.3.0 in /opt/conda/envs/Python36/lib/python3.6/site-packages (4.4.1)


In [5]:
# Add Watson Language Translators Service credentials for convert multiple language responses into English
from ibm_watson import LanguageTranslatorV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('Bl81vpxYkB5y6DlhpISunsyjS0zOYME2JWRD0xWnCbgt')
service = LanguageTranslatorV3(version='2018-05-01',
   authenticator=authenticator
)
service.set_service_url('https://api.us-south.language-translator.watson.cloud.ibm.com/instances/78b55083-ee3f-4517-a209-1f07d880000d')

In [6]:
# Saving Model ID's in list ex:(Hindi-English) 
models_list=[]
size= len((service.list_models().get_result()).get('models'))
for i in range(0,size):
    models_list.append((service.list_models().get_result()).get('models')[i].get('model_id'))

In [7]:
sql_data['Response'] = sql_data['Response'].astype(str)
sql_data.dtypes

Customer Type       object
Comments            object
Response            object
Categorisation      object
Sentiment           object
Sector              object
Flight No           object
Flight Date         object
Cabin Class         object
Seat No             object
Aircraft Version    object
Tail No             object
FFP Member          object
Gender              object
Age                 object
Country             object
dtype: object

In [8]:
from tqdm import tqdm

In [9]:
# Converting Multilanguage customer Responses to English
# saving outputs in List
identified_lang=[]
translated_lang=[]
identify_lang=list(sql_data['Response'])
for i in tqdm(identify_lang):
    detected_lang=(service.identify(i).get_result()).get('languages')[0].get('language')
    identified_lang.append(detected_lang)
    if (detected_lang != 'en' and '{}-en'.format(detected_lang) in models_list):
        tran=(service.translate(text=i,model_id='{}-en'.format(detected_lang)).get_result()).get('translations')[0].get('translation')
        translated_lang.append(tran)
    else:
        translated_lang.append(i)

100%|██████████| 5347/5347 [13:39<00:00,  7.21it/s]


In [10]:
# inserting Identified Language ID & Translated_Language in to Dataframe
sql_data.insert(2, "Identified_Language", identified_lang, True)
sql_data.insert(3, "Translated_Language", translated_lang, True)

In [11]:
# Checking Shape of dataframe(Number of records & Coulmns Available in Dataframe)
sql_data.shape

(5347, 18)

In [12]:
# Sample View of Dataframe
sql_data.head()

,Customer Type,Comments,Identified_Language,Translated_Language,Response,Categorisation,Sentiment,Sector,Flight No,Flight Date,Cabin Class,Seat No,Aircraft Version,Tail No,FFP Member,Gender,Age,Country
0,Passive,Amenity Kit: Comments,fr,Very well.,Très bien.,Compliment,Positive,MRU-CDG,NK0034,26/11/2018,ECONOMY,039E,359C,3BNBQ,NO,Male,26-34,France
1,Detractor,Amenity Kit: Comments,en,Not taking well kids,Not taking well kids,No adequate products,Negative,MRU-MAA,NK0746,26/11/2018,ECONOMY,None,343E,3BNBJ,NO,Male,35-44,Mauritius
2,Detractor,Amenity Kit: Comments,fr,Up to the detail we see the regression of Air ...,jusque dans le détail nous voyons la régressio...,Poor quality/ Very dissatisfied,Negative,BOM-MRU,NK0749,26/11/2018,ECONOMY,011C,343E,3BNBI,NO,Female,45-54,Mauritius
3,Detractor,Amenity Kit: Comments,en,I appreciate this kit in economy. It is a welc...,I appreciate this kit in economy. It is a welc...,Compliment,Positive,SIN-MRU,NK0647,27/11/2018,ECONOMY,016B,332,3BNBM,NO,Female,55-64,Mauritius
4,Detractor,Amenity Kit: Comments,en,Nothing,Nothing,None,Neutral,MRU-BOM,NK0748,27/11/2018,ECONOMY,026H,343C,3BNAU,YES,Male,45-54,India


In [13]:
# Import libraries for Sentiments extraction using Watson Language Understanding Service From IBM Cloud
#import pandas as pd
#import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, ConceptsOptions, EmotionOptions, SentimentOptions
from ibm_watson import ApiException
import operator

In [14]:
# Adding Watson Natural Language Understanding credentials 
authenticator = IAMAuthenticator('LHlV5n-FskBluBSFGPgNPt5SyLuw8rKklwrBOjE191sA')
natural_language_understanding = NaturalLanguageUnderstandingV1(version='2019-07-12',authenticator=authenticator
)

natural_language_understanding.set_service_url('https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/ae1b361c-624c-4289-bb6e-d0e5405d88d8')

In [15]:
# Dummy List's For Sentiment Extraction
sentiment=[]
sentiment_score=[]
keywords=[]
keywords_label=[]
keywords_relevance=[]


emotion_sadness=[]
emotion_joy=[]
emotion_fear=[]
emotion_disgust=[]
emotion_anger=[]


concepts_text=[]
concepts_relevance=[]

In [16]:
from tqdm import tqdm

In [17]:
for i in (range(0,len(sql_data['Translated_Language']))):
    try:
        response = natural_language_understanding.analyze(text=str(sql_data['Translated_Language'][i]),
        language='en',                          
        features=Features(
        entities=EntitiesOptions(emotion=True, sentiment=True, limit=1),
        keywords=KeywordsOptions(emotion=True, sentiment=True,limit=1),
        concepts=ConceptsOptions(limit=1),
        emotion=EmotionOptions(document=True),
        sentiment=SentimentOptions(document=True))).get_result()

     #sentiments list collection    
        try:
            (response['sentiment']['document']['label']) != ''  
            sentiment.append(response['sentiment']['document']['label'])   
        except:
            sentiment.append('')
        try:   
            response['sentiment']['document']['score'] != ''
            sentiment_score.append(response['sentiment']['document']['score'])    
        except:       
            sentiment_score.append('')

    #keywords list collections
        try:
            (response['keywords'][0]['text']) != ''       
            keywords.append((response['keywords'][0]['text']))
        except:        
            keywords.append('')
        try:        
            (response['keywords'][0]['sentiment']['label']) != ''     
            keywords_label.append(response['keywords'][0]['sentiment']['label'])
        except:        
            keywords_label.append('')

        try:           
            (response['keywords'][0]['relevance']) != ''         
            keywords_relevance.append(response['keywords'][0]['relevance'])
        except:        
            keywords_relevance.append('')

    #emotion extraction

        try:
            (response['emotion']) != []      
            emotion_sadness.append(response['emotion']['document']['emotion']['sadness'])
            emotion_anger.append(response['emotion']['document']['emotion']['anger'])
            emotion_joy.append(response['emotion']['document']['emotion']['joy'])
            emotion_disgust.append(response['emotion']['document']['emotion']['disgust'])
            emotion_fear.append(response['emotion']['document']['emotion']['fear'])

        except:
            emotion_anger.append('')
            emotion_disgust.append('')
            emotion_fear.append('')
            emotion_joy.append('')
            emotion_sadness.append('')

    #extracting concepts
        try:
            response['concepts'] != []
            concepts_text.append(response['concepts'][0]['text'])
            print(concepts_text)
            concepts_relevance.append(response['concepts'][0]['relevance'])
            print(concepts_relevance)
                
        except:
            
            concepts_relevance.append('')
            concepts_text.append('')
    except:
        print(False + i)
        sentiment.append('')
        sentiment_score.append('')
        keywords.append('')
        keywords_label.append('')
        keywords_relevance.append('')


        emotion_sadness.append('')
        emotion_joy.append('')
        emotion_fear.append('')
        emotion_disgust.append('')
        emotion_anger.append('')


        concepts_text.append('')
        concepts_relevance.append('')

  0%|          | 0/5347 [00:00<?, ?it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
  0%|          | 1/5347 [00:00<1:05:06,  1.37it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
  0%|          | 2/5347 [00:00<49:22,  1.80it/s]  

0
1


ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
  0%|          | 3/5347 [00:01<42:33,  2.09it/s]

2


ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
  0%|          | 4/5347 [00:01<39:57,  2.23it/s]

...

5342
5343


ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
100%|█████████▉| 5345/5347 [12:12<00:00,  7.53it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
100%|█████████▉| 5346/5347 [12:12<00:00,  7.41it/s]

5344
5345


ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
100%|██████████| 5347/5347 [12:12<00:00,  7.30it/s]

5346


In [18]:
# Making Dataframe using Sentiment Extraction List's
import pandas as pd
df=pd.DataFrame({'sentiment':sentiment,'sentiment_score':sentiment_score,'keywords':keywords,'keywords_label':keywords_label
                ,'keywords_relevance':keywords_relevance,'emotion_anger':emotion_anger,'emotion_disgust':emotion_disgust,
                'emotion_fear':emotion_fear,'emotion_joy':emotion_joy,'emotion_sadness':emotion_sadness,
                'concepts_text':concepts_text,'concepts_relevance':concepts_relevance})

In [19]:
# Sample View of Sentiments
df.head()

,sentiment,sentiment_score,keywords,keywords_label,keywords_relevance,emotion_anger,emotion_disgust,emotion_fear,emotion_joy,emotion_sadness,concepts_text,concepts_relevance
0,,,,,,,,,,,,
1,,,,,,,,,,,,
2,,,,,,,,,,,,
3,,,,,,,,,,,,
4,,,,,,,,,,,,


In [20]:
# Information of df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5347 entries, 0 to 5346
Data columns (total 12 columns):
sentiment             5347 non-null object
sentiment_score       5347 non-null object
keywords              5347 non-null object
keywords_label        5347 non-null object
keywords_relevance    5347 non-null object
emotion_anger         5347 non-null object
emotion_disgust       5347 non-null object
emotion_fear          5347 non-null object
emotion_joy           5347 non-null object
emotion_sadness       5347 non-null object
concepts_text         5347 non-null object
concepts_relevance    5347 non-null object
dtypes: object(12)
memory usage: 501.4+ KB


In [21]:
# Making duplicate dataframe
sql_data1=sql_data

In [22]:
frames=[sql_data1,df]

In [23]:
# Concating
result = pd.concat(frames, axis=1)

In [24]:
result.head()

,Customer Type,Comments,Identified_Language,Translated_Language,Response,Categorisation,Sentiment,Sector,Flight No,Flight Date,...,keywords,keywords_label,keywords_relevance,emotion_anger,emotion_disgust,emotion_fear,emotion_joy,emotion_sadness,concepts_text,concepts_relevance
0,Passive,Amenity Kit: Comments,fr,Very well.,Très bien.,Compliment,Positive,MRU-CDG,NK0034,26/11/2018,...,,,,,,,,,,
1,Detractor,Amenity Kit: Comments,en,Not taking well kids,Not taking well kids,No adequate products,Negative,MRU-MAA,NK0746,26/11/2018,...,,,,,,,,,,
2,Detractor,Amenity Kit: Comments,fr,Up to the detail we see the regression of Air ...,jusque dans le détail nous voyons la régressio...,Poor quality/ Very dissatisfied,Negative,BOM-MRU,NK0749,26/11/2018,...,,,,,,,,,,
3,Detractor,Amenity Kit: Comments,en,I appreciate this kit in economy. It is a welc...,I appreciate this kit in economy. It is a welc...,Compliment,Positive,SIN-MRU,NK0647,27/11/2018,...,,,,,,,,,,
4,Detractor,Amenity Kit: Comments,en,Nothing,Nothing,None,Neutral,MRU-BOM,NK0748,27/11/2018,...,,,,,,,,,,
